### Measuring transaction costs and trading volume

In [ ]:
import sys
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import scipy.stats as scipy
sys.path.append("utils")
import dynamic_backtesting as db
from compare_strategies import calc_transaction_costs as ctc
from compare_strategies import clean_up_returns
plt.style.use('seaborn')
matplotlib.rcParams['axes.prop_cycle'] = matplotlib.cycler(color=["black", "green",  "deepskyblue", "orange","grey", "lime", "purple", "Gold", "brown", "blue", "pink"]) 

In [ ]:
assets = {"EEM": "Emerging Markets",
           "IVV": "S&P 500",
           "IEV": "MSCI Europe",
           "IXN": "Global Tech",
           "IYR": "US Real Estate",
           "IXG": "Global Financials",
           "EXI": "Global Industrials",
           "GC=F": "Gold Futures", 
           "BZ=F": "Brent Crude Oil Futures",
           "HYG": "High-Yield Corporate Bonds",
           "TLT": "20+ Year Treasury Bonds"}
tickers = list(assets.keys())
asset_names = list(assets.values())
stocks = tickers[:7]
commodities = tickers[7:9]
bonds = tickers[9:]

In [ ]:
models = ['sGARCH10', 'sGARCH11', 'gjrGARCH11']

In [ ]:
def remove_extreme(num, tolerance):
    if num > tolerance:
        return tolerance
    if num < 0:
        return np.nan
    return num

In [ ]:
def calc_TC(model, assets):
    v_t_s, out_of_sample, in_sample, Omega_ts = db.garch_no_trading_cost(tickers=assets, model_type=model)
    out_of_sample = out_of_sample.divide(100)
    
    delayed_weights = v_t_s.shift(1).iloc[1:]
    portfolio_returns = delayed_weights.multiply(out_of_sample).sum(axis=1).to_frame()
    
    
    TC = ctc(v_t_s, out_of_sample, Omega_ts)
    portfolio_returns.columns = ['GARCH']
    portfolio_returns['GARCH TC'] = portfolio_returns['GARCH']-TC
    cum_portfolio_returns = portfolio_returns.add(1).cumprod()
    cum_portfolio_returns = cum_portfolio_returns.divide(cum_portfolio_returns.iloc[0])
    cum_portfolio_returns = cum_portfolio_returns.apply(clean_up_returns, axis=0)
    
    last_valid_index = cum_portfolio_returns.reset_index().loc[:, ['GARCH TC']].last_valid_index()+1
    print(f'Model: {model}\t Last valid index: {last_valid_index}\n')
    TC = TC[:last_valid_index]
    TC = [remove_extreme(x, 1) for x in TC]
    
    
    return TC

In [ ]:
def calc_multiple_TC(tickers):
    TCs = []
    for model in models:
        row = []
        row.append(calc_TC(model, tickers))
        TCs.append(row)
    TCs_mean = [[np.nanmean(TC) for TC in TC_row] for TC_row in TCs]
    return TCs, TCs_mean

In [ ]:
TCs_all, TCs_all_mean = calc_multiple_TC(tickers)

In [ ]:
TCs_bonds, TCs_bonds_mean = calc_multiple_TC(bonds)

In [ ]:
TCs_stocks, TCs_stocks_mean = calc_multiple_TC(stocks)

In [ ]:
TCs_com, TCs_com_mean = calc_multiple_TC(commodities)

In [ ]:
def annualize_TC(TC):
    TC = np.array(TC)*(-1)
    TC += 1
    TC_cum = np.cumprod(TC)

    total_costs = pd.Series(TC_cum).ffill().iloc[-1]
    non_num_nan = len(TC)-np.isnan(TC).sum()
    ann_TC = 1-(total_costs ** (250/non_num_nan))
    return ann_TC

In [ ]:
TC_all_ann = [[annualize_TC(TC) for TC in TC_row] for TC_row in TCs_all]

In [ ]:
TC_bonds_ann = [[annualize_TC(TC) for TC in TC_row] for TC_row in TCs_bonds]

In [ ]:
TC_stocks_ann = [[annualize_TC(TC) for TC in TC_row] for TC_row in TCs_stocks]

In [ ]:
TC_com_ann = [[annualize_TC(TC) for TC in TC_row] for TC_row in TCs_com]

In [21]:
TC_all_ann

[[1.0], [0.24144749602397064], [0.2713738008864348]]

In [22]:
TC_stocks_ann

[[1.0], [0.8394906663634426], [0.8377968540780192]]

In [ ]:
TCs_stocks[1]

### Equal weight costs

In [23]:
def calc_TC_EW():
    TCs = []
    assets = [tickers, bonds, stocks, commodities]
    for model in models:
        row = []
        for asset_list in assets:
            out_of_sample, in_sample, sigmas, residuals, params_dict = db.split_fit_parse(asset_list, start="2008-01-01", end="2021-10-02",
                                                                               number_of_out_of_sample_days=1000, model_type=model)
            Omega_ts = db.calc_Omega_ts(out_of_sample_returns=out_of_sample, in_sample_returns=in_sample,
                             in_sample_sigmas=sigmas, in_sample_residuals=residuals, **params_dict)
            weight_index = in_sample.index[[-1]].union(out_of_sample.index)
            
            p = len(asset_list)
            weights_shape = (1001, p)
            EW = pd.DataFrame(np.full(weights_shape, (1/p)), index=weight_index)
            out_of_sample = out_of_sample.divide(100)
            TC_EW = ctc(weights=EW, returns=out_of_sample, Omega_ts=Omega_ts, portfolio_value=1e9)
            TC_EW = [remove_extreme(x, 1) for x in TC_EW]
            row.append(TC_EW)
        TCs.append(row)
    return TCs

In [24]:
TCs_EW = calc_TC_EW()

['EEM', 'IVV', 'IEV', 'IXN', 'IYR', 'IXG', 'EXI', 'GC=F', 'BZ=F', 'HYG', 'TLT']
['HYG', 'TLT']
['EEM', 'IVV', 'IEV', 'IXN', 'IYR', 'IXG', 'EXI']
['GC=F', 'BZ=F']
['EEM', 'IVV', 'IEV', 'IXN', 'IYR', 'IXG', 'EXI', 'GC=F', 'BZ=F', 'HYG', 'TLT']
['HYG', 'TLT']
['EEM', 'IVV', 'IEV', 'IXN', 'IYR', 'IXG', 'EXI']
['GC=F', 'BZ=F']
['EEM', 'IVV', 'IEV', 'IXN', 'IYR', 'IXG', 'EXI', 'GC=F', 'BZ=F', 'HYG', 'TLT']
['HYG', 'TLT']
['EEM', 'IVV', 'IEV', 'IXN', 'IYR', 'IXG', 'EXI']
['GC=F', 'BZ=F']


In [25]:
TCs_EW_ann = [[annualize_TC(TC) for TC in TC_row] for TC_row in TCs_EW]

In [26]:
TCs_EW_ann

[[0.043599296043518865,
  0.008481584089645389,
  0.01312667288290481,
  0.06357385076324906],
 [0.05020255378691285,
  0.009966997484608497,
  0.014707796587701716,
  0.08225260437137927],
 [0.05237439235482899,
  0.011340533154792332,
  0.01498933624827048,
  0.0778527615801684]]